In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json


In [5]:
data = json.load(open('./data/train_data.json','r'))

In [6]:
data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [7]:

def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = set()

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue

            entity_indices.update(range(start, end))

            try:
                span = doc.char_span(start, end, label=label,alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = f"{[start, end]}  {text}\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db


In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data,test_size=0.2)

In [10]:
print(len(train),len(test))

160 40


In [11]:
file = open("error_log.txt", "w") 

ner_train = get_spacy_doc(file,train)
ner_train.to_disk('train_data.spacy')

ner_test = get_spacy_doc(file,test)
ner_test.to_disk('test_data.spacy')

file.close()

100%|██████████| 40/40 [00:00<00:00, 97.17it/s] 
